In [1]:
from ipywidgets import widgets
from IPython.display import display
from Levenshtein import distance
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pipeline import Pipeline
import requests
from PIL import Image
from io import BytesIO

In [2]:
input_text = widgets.Text(placeholder="Enter your favourite book")
output_text = widgets.Output()

In [3]:
def get_most_similar_title(title, books):
    global tfidf_vectorizer, tfidf_matrix, combined_data, mapping
    vectorized_input = tfidf_vectorizer.transform([title])
    similarities = cosine_similarity(tfidf_matrix, vectorized_input)
    row_index = similarities.argmax()
    closest_string = list(mapping.keys())[row_index]
    return closest_string

In [4]:

def on_click(sender):
    output_text.clear_output()
    global books
    global mapping
    with output_text:
        print("Most similar book in the archive:")
        matched_title = get_most_similar_title(str(input_text.value), books)
        print(matched_title)
        print("Top 5 similar books:")
        similar_books = mapping.get(matched_title)
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
        for book in similar_books:
            r = requests.get(books.loc[books.bookTitle == book,"Image-URL-M"].iloc[0], headers=headers)
            if r.status_code == 200:
                image_bytes = BytesIO(r.content)
                img = Image.open(image_bytes)
            print("-", book)
            display(img)

#load preprocessed data
books = pd.read_csv("./../data/books_filtered.csv")
books_with_urls = pd.read_csv("./../data/Books.csv")
combined_data = pd.read_csv("./../data/merged_data.csv")
mapping = Pipeline.get_item_item_similar_mapping()
tfidf_vectorizer = TfidfVectorizer(stop_words = "english", ngram_range = (1,5),analyzer = "char_wb")
tfidf_matrix = tfidf_vectorizer.fit_transform(list(mapping.keys()))
books = books.merge(books_with_urls[["ISBN","Image-URL-M"]],on="ISBN")
input_text.on_submit(on_click)
display(input_text)
display(output_text)

/tmp/ipykernel_30783/2837832672.py:22: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_with_urls = pd.read_csv("./../data/Books.csv")
/home/tomas_barhon/Code/book_recommendation_kaggle/book_recommendation/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:547: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/tmp/ipykernel_30783/2837832672.py:28: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_text.on_submit(on_click)


Text(value='', placeholder='Enter your favourite book')

Output()